In [1]:
import numpy as np
import pandas as pd
import os
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
access_token = os.getenv('HUGGINGFACE_TOKEN')

In [ ]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B", token=access_token)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a function to generate text
def generate_text(prompt, max_length=100):
    inputs = tokenizer.encode_plus(
        prompt,
        add_special_tokens=True,
        max_length=max_length,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Move the input tensors to the GPU
    inputs['input_ids'] = inputs['input_ids'].to(device)
    inputs['attention_mask'] = inputs['attention_mask'].to(device)

    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hello, how are you? I am so sorry that I didn’t post any recipes for a long time. I am very busy in my life. But I am very happy. I have my own blog now. It’s a very important step for me. I have a dream. I want to become a famous blogger. I hope I can do it. My dream is very big, but I have to do it. I am very excited. I hope you will enjoy my blog. Thank


In [5]:
# Test the function
prompt = "Give me a travel plan in Philadelphia for 2 days"
generated_text = generate_text(prompt)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Give me a travel plan in Philadelphia for 2 days. I have a car and I want to see all the historical places. I am coming from New York.
The Liberty Bell is a must see. The Constitution Center is a great place to learn about the Constitution. The National Constitution Center is the first museum in the United States dedicated to the U.S. Constitution and the story of We the People. The museum is located on the Delaware River in Historic Philadelphia, Pennsylvania. It is just a
